<a href="https://colab.research.google.com/github/sgsoul/applied-ai/blob/main/bonus_track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Предсказание отчисления студентов с потоков дополнительного образования Bonus Track**

Целью нашего итогового проекта является получение зависимости от различных параметров(Например, образовательная программа, ступень образования, курс обучения) отчисления студентов программы дополнительного образования Bonus Track, а также предсказание этого отчисления с помощью нейронной сети.

## Сбор данных

Изначально мы собрали информацию об отчисленных с помощью кода ниже. Мы получили список имён и образовательных программ. Потом рандомно добили датасет до 500 неотчисленными студентами и через ИСУ вручную собрали необходимую информацию.


In [ ]:
# import csv

# # открываем файлы содержащие спсики студентов на сентябрь 22 и февраль 23
# with open('first.csv', 'r') as f1, open('second.csv', 'r') as f2:
#     first_list = list(csv.reader(f1))
#     second_list = list(csv.reader(f2))

#     diff_list = []

#     for item in first_list:
#         if item not in second_list:
#             diff_list.append(item)

#     # записываем список отчисленных студентов в новый файл
# with open('third.csv', 'w') as f3:
#     writer = csv.writer(f3)
#     writer.writerows(diff_list)

Представленный датасет содержит описание набора 500 студентов Bonus Track’a. Он содержит следующие признаки:

1. Образовательная программа (Технологии анализа данных - 0, Методы анализа данных - 1)
2. Год обучения в ИТМО (1-4 курсы)
3. Степень образования (бакалавр - 0, магистр - 1)
4. Факультет в ИТМО:
    1. БИТ - Факультет Безопасности информационных технологий,
    2. ВИТШ - Институт Высшая инженерно-техническая школа,
    3. ИДУ - Институт Дизайна и урбанистики,
    4. ИКТ - Факультет Инфокоммуникационных технологии,
    5. ИЛТ - Институт Лазерных технологий,
    6. ИПСПД - Институт Перспективных систем передачи данных,
    7. ИТИП - Факультет Информационных технологий и программирования,
    8. МРИП - Институт Международного развития и партнерства,
    9. ПИИКТ - Факультет Программной инженерии и компьютерной техники,
    10. СУИР - Факультет Систем управления и робототехники,
    11. ФБТ - Факультет Биотехнологий,
    12. ФИЗФ - Физический факультет,
    13. ФИОИ - Фотоника и оптоинформатика,
    14. ФТМИ - Факультет Технологического менеджмента и инноваций,
    15. ФТМФ - Физико-Технический мегафакультет,
    16. ФТФ - Физико-Технический факультет,
    17. ФЭТ - Факультет Экотехнологий,
    18. ХБК - Химико-Биологический кластер,
    19. ХИ - Центр химической инженерии,
    20. ЦПО - Центр дополнительного профессионального образования,
    21. ЭИС - Образовательный центр Энергоэффективные инженерные системы
5. Статус студента (обучается - 0, отчислен - 1)

## Реализация

Импортируем необходимые нам библиотеки и методы.

In [ ]:
!pip install category_encoders -q
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.svm import SVC
import category_encoders as ce
from tensorflow import keras
from tensorflow.keras import layers, callbacks, backend, optimizers

Считываем данные, предварительно загруженные в среду блокнота. Удаляем столбик 'ID', так он нам не пригодится в процессе обучения.



In [ ]:
# Чтение данных из CSV файла
df = pd.read_csv('/content/TRAIN.csv')
df.describe()

,ID,BT_PROG,YEAR,DEGREE,STATUS
count,450.000000,450.000000,450.000000,450.000000,450.000000
mean,225.500000,0.526667,2.224444,0.295556,0.415556
std,130.048068,0.499844,0.929989,0.537442,0.493366
min,1.000000,0.000000,1.000000,0.000000,0.000000
25%,113.250000,0.000000,2.000000,0.000000,0.000000
50%,225.500000,1.000000,2.000000,0.000000,0.000000
75%,337.750000,1.000000,3.000000,1.000000,1.000000
max,450.000000,1.000000,4.000000,4.000000,1.000000


In [ ]:
df = df.drop('ID', axis=1)

Разделяем данные на обучающую, валидационную и тестовую выборки.

In [ ]:
# Разделение данных на обучающую, валидационную и тестовую выборки с учетом стратификации
train_data, test_data = train_test_split(df, test_size=0.2, random_state=35, stratify=df['STATUS'])
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=35, stratify=train_data['STATUS'])

Производим подготовку данных:


1.   Определяем категориальные признаки в `cat_features`
2.   Примененяем кодировщика `TargetEncoder` из библиотеки `category_encoders` для преобразования категориальных признаков в числовые
3. Масштабируем данные с помощью `StandardScaler` из библиотеки `sklearn.preprocessing`
4. Подготавливаем целевые переменные



In [ ]:
# Подготовка данных
cat_features = ['BT_PROG', 'YEAR', 'DEGREE', 'FACULTY']

encoder = ce.TargetEncoder(cols=cat_features)
train_data_encoded = encoder.fit_transform(train_data.drop('STATUS', axis=1), train_data['STATUS'])
val_data_encoded = encoder.transform(val_data.drop('STATUS', axis=1))
test_data_encoded = encoder.transform(test_data.drop('STATUS', axis=1))

scaler = StandardScaler()
train_data_encoded[cat_features] = scaler.fit_transform(train_data_encoded[cat_features])
val_data_encoded[cat_features] = scaler.transform(val_data_encoded[cat_features])
test_data_encoded[cat_features] = scaler.transform(test_data_encoded[cat_features])

# Подготовка целевых переменных
y_train = train_data['STATUS']
y_val = val_data['STATUS']
y_test = test_data['STATUS']

Вычисляем веса классов для взвешивания на основе дисбаланса классов в обучающей выборке.

In [ ]:
# Вычисление весов классов для взвешивания
class_weights = dict(1 / train_data['STATUS'].value_counts())
class_weights

{0: 0.005952380952380952, 1: 0.008333333333333333}

Производим определение метрик `recall_m`, `precision_m` и `f1_m`, которые используются при компиляции модели.

In [ ]:
# Определение метрик f1, precision и recall
def recall_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + backend.epsilon()))

Создаём модель с использованием `keras.Sequential`, в которой определены слои `Dense` с активацией `relu`. И компилируем модель с оптимизатором `Adam`, функцией потерь `МАЕ` и нашими метриками.

In [ ]:
# Создание модели
model = keras.Sequential([
    layers.InputLayer(input_shape=(4)),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='tanh')
])

# Компиляция модели с весами классов
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss='MAE',
    metrics=[f1_m, precision_m, recall_m]
)

Обучаем модель с использованием данных обучающей и валидационной выборок, определенных метрик и весов классов.

Также создаём обратный вызов `EarlyStopping`, который остановит обучение, если метрика `val_f1_m` не улучшится в течение 3 эпох, и обратный вызов `ModelCheckpoint`, который сохраняет лучшую модель с наилучшей метрикой `val_f1_m`.

In [ ]:
# Обучение модели с весами классов
es_callback = callbacks.EarlyStopping(patience=3)
filename = 'model.h5'
checkpoint = callbacks.ModelCheckpoint(filename, monitor='val_f1_m', verbose=1, save_best_only=True, mode='max')

history = model.fit(
    train_data_encoded,
    y_train,
    epochs=30,
    batch_size=64,
    callbacks=[checkpoint, es_callback],
    validation_data=(val_data_encoded, y_val),
    class_weight=class_weights
)

Epoch 1/30
3/5 [=================>............] - ETA: 0s - loss: 0.0041 - f1_m: 0.1500 - precision_m: 0.1017 - recall_m: 0.2857             
Epoch 1: val_f1_m improved from -inf to 0.50350, saving model to model.h5
5/5 [==============================] - 4s 166ms/step - loss: 0.0037 - f1_m: 0.3257 - precision_m: 0.3062 - recall_m: 0.4132 - val_loss: 0.5548 - val_f1_m: 0.5035 - val_precision_m: 0.4066 - val_recall_m: 0.6635
Epoch 2/30
1/5 [=====>........................] - ETA: 0s - loss: 0.0025 - f1_m: 0.6774 - precision_m: 0.5833 - recall_m: 0.8077
Epoch 2: val_f1_m did not improve from 0.50350
5/5 [==============================] - 0s 21ms/step - loss: 0.0025 - f1_m: 0.6389 - precision_m: 0.5513 - recall_m: 0.7654 - val_loss: 0.5253 - val_f1_m: 0.5022 - val_precision_m: 0.4143 - val_recall_m: 0.6442
Epoch 3/30
5/5 [==============================] - ETA: 0s - loss: 0.0023 - f1_m: 0.6509 - precision_m: 0.5639 - recall_m: 0.7799
Epoch 3: val_f1_m improved from 0.50350 to 0.51801, saving

Загрузка лучшей модели с помощью `keras.models.load_model`. Предсказываем целевые переменные для тестовой выборки и вычисляем F1-меру с использованием функции `f1_score` из библиотеки `sklearn.metrics`.

In [ ]:
# Загрузка лучшей модели
model = keras.models.load_model('model.h5', custom_objects={'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})

# Предсказание и вычисление F1-меры
predictions = model.predict(test_data_encoded)
f1 = f1_score(y_test, predictions >= 0.8)
print(f"F1-score: {f1}")

3/3 [==============================] - 0s 10ms/step
F1-score: 0.5714285714285715


Загружаем зарезервироанные данные для предсказания отчисления студентов. Также отбрасываем столбец 'ID'.

In [ ]:
df1 = pd.read_csv('/content/PRED.csv')
df_new = df1.drop(['ID', 'STATUS'], axis = 1 )


Преобразуем категориальные признаки в числовые и масштабируем данные.

In [ ]:
ds_pred = encoder.transform(df_new)
ds_pred[cat_features] = scaler.transform(ds_pred[cat_features])

Выполним предсказания для зарезервированной части.

In [ ]:
pred_platform = model.predict(ds_pred[cat_features])
predictions = np.array([1 if pred_platform[_] >= 0.8 else 0 for _ in range(len(pred_platform))])
predictions

2/2 [==============================] - 0s 6ms/step


array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1])

Выведем более информативное отображение результата предсказания.

In [ ]:
output = pd.DataFrame({'ID':df1['ID'],'STATUS': np.squeeze(predictions)})
output

,ID,STATUS
0,451,1
1,452,1
2,453,0
3,454,1
4,455,1
5,456,1
6,457,1
7,458,1
8,459,1
9,460,1


## Метод опорных векторов

Теперь обучим модель `SVC()` на тренировочном наборе данных с подобранными параметрами, чтобы сравнить и подобрать лучшую модель для наших данных.

In [ ]:
# Создание модели SVM
model = SVC(random_state=42, gamma = 0.001, kernel ='rbf', C = 10, class_weight = 'balanced')

# Обучение модели
model.fit(train_data_encoded, y_train)

# Предсказание и вычисление F1-меры на тестовых данных
predictions = model.predict(test_data_encoded)
f1 = f1_score(y_test, predictions)

print("F1-score:", f1)

F1-score: 0.5833333333333333


Используя `GridSearchCV()`, осуществляем подбор гиперпараметров.

In [ ]:
# from sklearn.model_selection import GridSearchCV


# tuned_parameters = [{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000, 10000], 'class_weight': [None, 'balanced'], 'random_state':[42]}]


# cv = GridSearchCV(SVC(), tuned_parameters, refit=True, verbose=3).fit(train_data_encoded, y_train)


Выводим лучшие параметры после обучения.

In [ ]:
#cv.best_params_

Подставляем "лучшие параметры" в модель и смотрим на полученное значение F1- меры.

In [ ]:
# Создание модели SVM
model = SVC(class_weight='balanced', random_state=42, gamma = 'scale', C = 100)

# Обучение модели
model.fit(train_data_encoded, y_train)

# Предсказание и вычисление F1-меры на тестовых данных
predictions = model.predict(test_data_encoded)
f1 = f1_score(y_test, predictions)

print("F1-score:", f1)

F1-score: 0.6097560975609757


Показания улучшились! Теперь предскажем наши отчисления студентов по этой модели.

In [ ]:
pred_platform = model.predict(ds_pred[cat_features])
predictions = np.array([1 if pred_platform[_] >= 0.8 else 0 for _ in range(len(pred_platform))])
predictions

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0])

##Свёрточная нейронная сеть

Теперь попробуем обучить нашу модель с помощью свёрточной нейронной сети. Для начала загрузим необходимые библиотеки.

In [ ]:
import torchvision
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import f1_score

# Создание модели
model = Sequential()
model.add(Conv1D(1024, kernel_size=3, activation='PReLU', input_shape=(4, 1)))  # Update input shape here
model.add(MaxPooling1D(pool_size=2))  # Update pool_size here
model.add(Flatten())
model.add(Dense(512, activation='PReLU'))
model.add(Dense(512, activation='PReLU'))
model.add(Dense(256, activation='PReLU'))
model.add(Dense(256, activation='PReLU'))
model.add(Dense(128, activation='PReLU'))
model.add(Dense(128, activation='PReLU'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh'))


# Компиляция модели
model.compile(optimizer='adam', loss='MAE', metrics=['accuracy'])

# В данном случае данные будут разделены на блоки по 3 значения, создавая 286 сэмплов, каждый с 3 временными шагами и 1 признаком
data = df.values
data = np.reshape(data, (data.shape[0], data.shape[1], 1))

X_train = train_data_encoded
X_test = test_data_encoded

# Обучение модели
model.fit(X_train, y_train, epochs= 10, batch_size=64, validation_data=(X_test, y_test))

# Предсказания на тестовых данных
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.8).astype(int)

# Вычисление F1-метрики
f1 = f1_score(y_test, y_pred)

print("F1 Score:", f1)


Epoch 1/10
5/5 [==============================] - 5s 268ms/step - loss: 0.4218 - accuracy: 0.5833 - val_loss: 0.4233 - val_accuracy: 0.5889
Epoch 2/10
5/5 [==============================] - 1s 116ms/step - loss: 0.4159 - accuracy: 0.5868 - val_loss: 0.3895 - val_accuracy: 0.5889
Epoch 3/10
5/5 [==============================] - 0s 104ms/step - loss: 0.3967 - accuracy: 0.6285 - val_loss: 0.3697 - val_accuracy: 0.6556
Epoch 4/10
5/5 [==============================] - 1s 112ms/step - loss: 0.3638 - accuracy: 0.6458 - val_loss: 0.3480 - val_accuracy: 0.6778
Epoch 5/10
5/5 [==============================] - 1s 116ms/step - loss: 0.3470 - accuracy: 0.6493 - val_loss: 0.4143 - val_accuracy: 0.6111
Epoch 6/10
5/5 [==============================] - 0s 89ms/step - loss: 0.3317 - accuracy: 0.6875 - val_loss: 0.3574 - val_accuracy: 0.6444
Epoch 7/10
5/5 [==============================] - 0s 96ms/step - loss: 0.3202 - accuracy: 0.6944 - val_loss: 0.3511 - val_accuracy: 0.6556
Epoch 8/10
5/5 [======

In [ ]:
pred_platform = model.predict(ds_pred[cat_features])
predictions = np.array([1 if pred_platform[_] >= 0.8 else 0 for _ in range(len(pred_platform))])
predictions

2/2 [==============================] - 0s 17ms/step


array([0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0])

## Классификатор MLPClassifier

Тут мы решили попробовать самый простой известный нам вариант - просто обучить классификатор и всё!)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Обучение
classifier = MLPClassifier(random_state=65, hidden_layer_sizes=(31, 10), activation='logistic', max_iter=1000)
classifier.fit(train_data_encoded, y_train)

# Оценка модели на тестовых данных
y_pred = classifier.predict(test_data_encoded)
f1 = f1_score(y_test, y_pred, average='macro')

# Вывод f1 метрики
print("F1-score:", f1)

F1-score: 0.6309458720612356


Вуа-ля! Самый высокий результат на этих данных готов.

In [ ]:
pred_platform = classifier.predict(ds_pred[cat_features])
predictions = np.array([1 if pred_platform[_] >= 0.8 else 0 for _ in range(len(pred_platform))])
predictions

array([0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0])

## Получение зависимости отчисления от различных параметров

In [ ]:
# Группировка данных по образовательной программе, ступени образования и курсу обучения
grouped_data = df.groupby(['BT_PROG', 'DEGREE', 'YEAR', 'FACULTY'])

# Подсчет количества отчисленных студентов в каждой группе
dropout_count = grouped_data['STATUS'].sum()

# Подсчет общего количества студентов в каждой группе
total_count = grouped_data['STATUS'].count()

# Вычисление доли отчисленных студентов в каждой группе
dropout_rate = dropout_count / total_count

# Сортировка групп по убыванию доли отчисленных студентов
sorted_groups = dropout_rate.sort_values(ascending=False)

# Вывод топ-N групп с наивысшей долей отчисленных студентов
top_n = 30
top_groups = sorted_groups.head(top_n)
print("Топ-{} групп с наивысшей долей отчисленных студентов:".format(top_n))
print(top_groups)


Топ-30 групп с наивысшей долей отчисленных студентов:
BT_PROG  DEGREE  YEAR  FACULTY
1        1       1     ХИ         1.000000
0        0       2     ЭИС        1.000000
         1       2     ФТМИ       1.000000
1        0       2     ИЛТ        1.000000
0        1       1     МРИП       1.000000
                       ИЛТ        1.000000
                       БИТ        1.000000
         0       4     ИЛТ        1.000000
                       ИКТ        1.000000
1        0       4     ИТИП       1.000000
0        0       3     ИТИП       1.000000
                 4     ФТМИ       1.000000
1        1       1     ИКТ        1.000000
0        0       2     ФЭТ        1.000000
                       ИТИП       1.000000
                       ФБТ        0.857143
                       ФИЗФ       0.800000
1        0       2     ИТИП       0.782609
0        0       2     ЦПО        0.666667
                 3     ПИИКТ      0.666667
         1       2     ФБТ        0.666667
1        0  

Тут мы учитываем факультет студентов, однако в силу того что многие вариации атрибутов были представлены только в одном студенте, мы получили результат в котором 15 наборов отчислятся в 100% случаях. Это не совсем корректно, поэтому попробуем не учитывать факультеты студентов и посмотрим на результаты.

In [ ]:
# Группировка данных по образовательной программе, ступени образования и курсу обучения
grouped_data = df.groupby(['BT_PROG', 'DEGREE', 'YEAR'])

# Подсчет количества отчисленных студентов в каждой группе
dropout_count = grouped_data['STATUS'].sum()

# Подсчет общего количества студентов в каждой группе
total_count = grouped_data['STATUS'].count()

# Вычисление доли отчисленных студентов в каждой группе
dropout_rate = dropout_count / total_count

# Сортировка групп по убыванию доли отчисленных студентов
sorted_groups = dropout_rate.sort_values(ascending=False)

# Вывод топ-N групп с наивысшей долей отчисленных студентов
top_n = 5
top_groups = sorted_groups.head(top_n)
print("Топ-{} групп с наивысшей долей отчисленных студентов:".format(top_n))
print(top_groups)

Топ-5 групп с наивысшей долей отчисленных студентов:
BT_PROG  DEGREE  YEAR
1        1       1       0.750000
0        0       4       0.600000
                 2       0.596154
1        0       4       0.512821
                 3       0.507246
Name: STATUS, dtype: float64


Получаем, такой список для самых частых отчислений:

1.   Методы анализа данных, магистры, 1 курс - 75.0%
2.   Технологии анализа данных, бакалавры, 4 курс - 60.0%
3.   Технологии анализа данных, бакалавры, 2 курс - 59.6%
4.   Методы анализа данных, бакалавры, 4 курс - 51.2%
3.   Методы анализа данных, бакалавры, 3 курс - 50.7%





## Выводы

Обучая нейронные сети для предсказаний мы добивались разных резултатов. Где-то они были выше, а где-то значительно ниже. Самый высокий результат на данный момент (0.63) был достигнут с помощью алгоритма многослойного перцептрона. К сожалению, результаты всё ещё не настолько точные, какими мы бы хотели их видеть, но это уже проблема ограниченности нашего датасета:( Подобрать ещё одни атрибуты у нас нет возможности и дата сама по себе не очень большая.

Что же касается статистики, то чаще всего отчисляются магистры "методов" на первом курсе.



---

Алексеева Елизавета, К32202

Лоскутова Ирина, К32202

Рогозина Вероника, К32202